In [ ]:

import firebase_admin
from firebase_admin import credentials, firestore


service_account_path = "/Users/narayanpowderly/Documents/atopile-workspace/package-registry-backend/atopile-880ca67acfe2.json"

if not firebase_admin._apps:  # Check if already initialized to prevent reinitialization
    cred = credentials.Certificate(service_account_path)
    firebase_admin.initialize_app(cred)


db = firestore.client()
_components_db = db.collection(
    "component_data"
)  

In [ ]:
# get all the inductors
db = firestore.client()
unparsed_parts = db.collection(
    "component_data"
)

# convert to dict
results = unparsed_parts.stream()
components = [doc.to_dict() for doc in results]

In [ ]:
import json


def get_resistor(req_json, firestore_client):
    # Get the component requirements
    min_value = req_json.get("min_value")
    max_value = req_json.get("max_value")
    package = req_json.get("package")

    # Query for resistors in the Firestore collection
    query = firestore_client.collection("components").where("type", "==", "Resistor")

    # Execute the query and get results
    results = query.stream()
    components = [doc.to_dict() for doc in results]
    print(len(components))

    #Strip leading 'R' from the package
    package = package[1:]

    # Filter the components by the requirements
    filtered_components = []
    for component in components:
        if min_value <= component["min_value"] and max_value >= component["max_value"] and component["Package"] == package:
            filtered_components.append(component)


    # Remove any that are out of stock
    filtered_components = [component for component in filtered_components if component["Stock"] > 100]

    # Find the cheapest component
    best_resistor = min(filtered_components, key=lambda component: component["Price (USD)"])

    # Send back a JSON response with the cheapest resistor
    return json.dumps(best_resistor)

In [ ]:
payload = {
    "type": "Resistor",
    "min_value": 900,
    "max_value": 1100,
    "package": "R0805"
}

get_resistor(payload, db)

In [ ]:
def get_mosfet(req_json, firestore_client):
# example json:
# {
#     "polarity": "N Channel",
#     "drain_source_voltage_V": 5,
#     "current_A": 0.1,
#     "on_resistance_ohms": "0.1",
#     "power_dissipation_W": "23"
# }
    query = firestore_client.collection("mosfets").where("type", "==", req_json["polarity"])
    results = query.stream()
    components = [doc.to_dict() for doc in results]
    # print(len(components))
    print(components[0])

    filtered_components = []
    for component in components:
        if component["drain_source_voltage_V"] >= req_json["drain_source_voltage_V"] and component["current_A"] >= req_json["current_A"]:# and component["on_resistance_ohms"] <= req_json["on_resistance_ohms"] and component["power_dissipation_W"] >= req_json["power_dissipation_W"]:
            filtered_components.append(component)
    print(len(filtered_components))
    best_mosfet = min(filtered_components, key=lambda component: component["Price"][0]["price"])



In [ ]:
#

query = {
    "polarity": "N Channel",
    "drain_source_voltage_V": "30",
    "current_A": "100",
    "on_resistance_ohms": "0.1",
    "power_dissipation_W": "23"
}

get_mosfet(query, db)

In [ ]:
# Reference to the collection
mosfets_collection = db.collection('mosfets')

# Fetch documents
docs = mosfets_collection.stream()

for doc in docs:
    doc_data = doc.to_dict()

    # Convert numeric fields from string to actual numbers
    doc_data['Stock'] = int(doc_data['Stock'])
    doc_data['current_A'] = float(doc_data['current_A'])
    doc_data['on_resistance_ohms'] = float(doc_data['on_resistance_ohms'])
    doc_data['drain_source_voltage_V'] = float(doc_data['drain_source_voltage_V'])
    doc_data['power_dissipation_W'] = float(doc_data['power_dissipation_W'])
    for price in doc_data['Price']:
        price['price'] = float(price['price'])

    # Rename 'type' to 'polarity' and add new 'type'
    doc_data['polarity'] = doc_data.pop('type')
    doc_data['type'] = 'mosfet'

    # Update the document
    mosfets_collection.document(doc.id).set(doc_data)

## Inductor Specifications

| Specification | Type | Example |
|---------------|------|---------|
| Rated Current | Physical type, unit Ampere | 3.45A |
| Inductance | Physical type, unit Henry | 10uH |
| Type | Categorical, specific type of inductor | Shielded Power Inductors |
| Tolerance | Percentage | ±20% |
| DC Resistance (DCR) | Physical type, unit Ohm | 25mΩ |
| Saturation Current (Isat) | Physical type, unit Ampere | 5.1A |

### ato example

inductor = new Inductor

inductor.current = 3.45A to 10A

inductor.inductance = 10uH +/- 30%

inductor.resistance = 0 to 25mohm

inductor.saturation_current = 5.1A to 10A

In [ ]:
# get all the inductors
db = firestore.client()
unparsed_parts = db.collection(
    "unparsed-parts"
)

# convert to dict
results = unparsed_parts.stream()
components = [doc.to_dict() for doc in results]

In [ ]:
# filter out inductors where "Description" contains "Power Inductors"
diodes = []
try:
    for component in components:
        if "Diode" in component["Description"]:
            diodes.append(component)

    # print out the first 10
    for i in range(0,10):
        print(diodes[i]["Description"])
        # extract_attribute_data
        pass
except:
    print("error")

In [ ]:
import re

def parse_description(description):
    # Regular expression patterns
    current_pattern = r'(\d+\.?\d*)A'
    inductance_pattern = r'(\d+\.?\d*uH)'
    tolerance_pattern = r'±(\d+)%'
    type_pattern = r'(SMD Power Inductors|Chip Resistor - Surface Mount)'
    resistor_value_pattern = r'(\d+\.?\d*KOhms)'
    
    # Extracting data
    current = re.search(current_pattern, description)
    inductance = re.search(inductance_pattern, description)
    tolerance = re.search(tolerance_pattern, description)
    component_type = re.search(type_pattern, description)
    resistor_value = re.search(resistor_value_pattern, description)

    # Formatting data into a dictionary
    data = {
        "current": current.group(1) + " Ampere" if current else None,
        "inductance": inductance.group(1) if inductance else None,
        "tolerance": "±" + tolerance.group(1) + "%" if tolerance else None,
        "type": component_type.group(1) if component_type else None,
        "resistor_value": resistor_value.group(1) if resistor_value else None
    }

    return data

# Example usage
descriptions = [
    "3.7A 3.3uH ±20% SMD Power Inductors ROHS",
    "3.3A 4.7uH ±20% SMD Power Inductors ROHS",
    "Chip Resistor - Surface Mount 4.7KOhms ±5% 1/16W 0402 RoHS"
]

for desc in descriptions:
    parsed_data = parse_description(desc)
    print(parsed_data)


In [ ]:
Stock -> stock
Price -> price
Description -> description
current 
resistance
saturation_current
tolerance
footprint
footprint_data["kicad"]


In [ ]:
import requests
import re

URL = "https://jlcpcb.com/partdetail/C9185"
page = requests.get(URL)


def extract_attribute_data(html):
    """
    Extracts attribute data from the given HTML string and returns it as a dictionary.

    Args:
    html (str): A string containing the HTML content.

    Returns:
    dict: A dictionary containing the extracted attribute names and values.
    """
    # Define the pattern to match the specified structure
    pattern = r'\{(?:\s*attribute_name_en:\s*"([^"]+)"\s*,\s*attribute_value_name:\s*"([^"]+)"|' + \
              r'\s*attribute_value_name:\s*"([^"]+)"\s*,\s*attribute_name_en:\s*"([^"]+)")\s*\}'

    # Find all matches in the HTML content
    matches = re.findall(pattern, html)

    # Create a dictionary from the matches
    extracted_data = {}
    for match in matches:
        # Match could be in either of the two capturing groups, depending on the order in HTML
        attribute_name = match[0] if match[0] else match[3]
        attribute_value = match[1] if match[1] else match[2]
        extracted_data[attribute_name] = attribute_value

    return extracted_data

print(extract_attribute_data(page.text))

In [ ]:
def extract_product_details(html):
    """
    Extracts product details from the given HTML string based on the provided structure.

    Args:
    html (str): A string containing the HTML content.

    Returns:
    dict: A dictionary containing the extracted product details.
    """
    # A dictionary to hold the extracted details
    details = {}

    # Regular expression pattern to match the details
    pattern = r'<dt[^>]*>(.*?)</dt>\s*<dd[^>]*>(.*?)</dd>'

    # Find all matches in the HTML content
    matches = re.findall(pattern, html, re.DOTALL)

    # Process each match
    for match in matches:
        key = match[0].strip()
        value = re.sub('<[^<]+?>', '', match[1]).strip()  # Remove any HTML tags from the value
        details[key] = value

    return details

print(extract_product_details(page.text))

In [ ]:
def extract_stock(html_string):
    # Regular expression pattern to find stock level
    stock_pattern = r'Available Order Qty: (\d+)'

    # Search for the pattern in the HTML string
    match = re.search(stock_pattern, html_string)

    # Extract and return stock level if found, else return 0
    if match:
        return int(match.group(1))
    else:
        return 0

print(extract_stock(page.text))

In [ ]:
def extract_pricing_details(html):
    """
    Extracts pricing details from the given HTML string using a revised approach.

    Args:
    html (str): A string containing the HTML content.

    Returns:
    str: The extracted price, or an empty string if not found.
    """
    # Regular expression pattern to match the price
    pattern = r'class="w-120 text-left text-2B8CED">\s*\$(\d+\.\d+)'

    # Find the first match in the HTML content
    match = re.search(pattern, html, re.DOTALL)

    if match:
        return match.group(1).strip()
    else:
        return ""



print(extract_pricing_details(page.text))

In [ ]:
from bs4 import BeautifulSoup

# Parse the HTML content
soup = BeautifulSoup(page.text, 'html.parser')

# Extracting price information
price_divs = soup.find_all("div", class_="flex items-center justify-between mt-14")
price = []

for div in price_divs:
    # Extract quantity and price
    qty = div.find("span", class_="w-120 inline-block").text.strip()
    pr = div.find("span", class_="w-120 text-left").text.strip().replace('$', '')

    # Add to the list as a tuple
    price.append((qty, float(pr)))

print(price[0][1])

In [ ]:
from tools import calculate_physical_properties
from pint import UnitRegistry

def parse_physical_properties(attributes):
    """
    Parses the attributes dictionary and calculates physical properties.

    Args:
    attributes (dict): Dictionary of attributes like 'Rated Current', 'Inductance', etc.

    Returns:
    dict: Dictionary containing physical properties.
    """
    ureg = UnitRegistry()
    properties = {}

    for key, value in attributes.items():
        if key == 'Inductance' and 'Tolerance' in attributes:
            # Extract nominal value and tolerance
            nominal_value, unit = re.match(r"([\d\.]+)(\w+)", value).groups()
            tolerance = float(re.search(r"±([\d\.]+)%", attributes['Tolerance']).group(1))

            # Calculate properties
            properties.update(calculate_physical_properties(value=float(nominal_value), tolerance=tolerance, unit=unit))

        elif key == 'Rated Current':
            # Assuming max value is the rated current and min value is 0
            max_val, unit = re.match(r"([\d\.]+)(\w+)", value).groups()
            properties.update(calculate_physical_properties(min_val=0, max_val=float(max_val), unit=unit))

    return properties

# Example usage:
attributes = {'Rated Current': '3.45A', 'Inductance': '10uH', 'Type': 'Shielded Power Inductors', 'Tolerance': '±20%', 'DC Resistance (DCR)': '25mΩ', 'Saturation Current (Isat)': '5.1A'}
physical_properties = parse_physical_properties(attributes)
print(physical_properties)


In [ ]:
# from tools import process_physical_attribute, generate_uuid
import tools

def create_inductor_data(html):
    attributes = extract_attribute_data(html)
    details = extract_product_details(html)

    inductor_data = {
        'uuid': tools.generate_uuid(),
        'inductance': tools.process_physical_attribute(float(re.match(r"([\d\.]+)(\w+)", attributes['Inductance']).groups()[0]), "uH", float(re.search(r"±([\d\.]+)%", attributes['Tolerance']).group(1))),
        'current': tools.process_physical_attribute(float(re.match(r"([\d\.]+)(\w+)", attributes['Rated Current']).groups()[0]), "A", min_val=0),
        'saturation_current': process_physical_attribute(float(re.match(r"([\d\.]+)(\w+)", attributes['Saturation Current (Isat)']).groups()[0]), "A", min_val=0),
        'resistance': process_physical_attribute(float(re.match(r"([\d\.]+)(\w+)", attributes['DC Resistance (DCR)']).groups()[0]), "ohm", tolerance=10),
        'manufacturer': details['Manufacturer'],
        'mpn': details['MFR.Part #'],
        'lcsc_id': details['JLCPCB Part #'],
        'package': details['Package'],
        'description': details['Description'],
        'stock': extract_stock(html),
        'price': extract_stock(html),
    }

    return inductor_data

In [ ]:
import tools

# get component data
page = tools.get_html(lcsc_id="C9185")

# check type:
# tools.get_type(page.text)


create_inductor_data(page)

In [ ]:
import glob
import re
import subprocess
import uuid
from pathlib import Path

import requests
from bs4 import BeautifulSoup
from pint import UnitRegistry

ureg = UnitRegistry()


def get_footprint_data(jlcpn):
    dir = "/Users/narayanpowderly/Documents/atopile-workspace/package-registry-backend/temp"
    command = [
        "easyeda2kicad",
        "--full",
        f"--lcsc_id={jlcpn}",
        f"--output={dir}/temp",
        "--overwrite",
        "--ato",
        f"--ato_file_path={dir}",
    ]
    result = subprocess.run(command, capture_output=True, text=True)
    # read the ato file data glob matching with .ato extension
    ato_file_path = glob.glob(f"{dir}/*.ato")[0]

    # read the kicad_mod file data
    kicad_mod_file_path = glob.glob(f"{dir}/temp.pretty/*.kicad_mod")[0]

    # Read the contents of the .ato and .kicad_mod files
    ato_file_content = read_file(ato_file_path)
    kicad_mod_file_content = read_file(kicad_mod_file_path)

    # Parse the ato file and transform the kicad_mod file
    ato_mapping = parse_ato_file(ato_file_content)
    transformed_kicad_mod = transform_kicad_mod(kicad_mod_file_content, ato_mapping)

    return transformed_kicad_mod


def read_file(file_path):
    with open(file_path, "r") as file:
        return file.read()


def parse_ato_file(ato_content):
    signal_mapping = {}
    for line in ato_content.splitlines():
        if "~ pin" in line:
            parts = line.strip().split(" ~ pin ")
            signal = parts[0].split(" ")[-1]
            pin = "pin " + parts[1]
            if signal not in signal_mapping:
                signal_mapping[signal] = []
            signal_mapping[signal].append(pin)
    return signal_mapping


def transform_kicad_mod(kicad_mod_content, ato_mapping):
    desired_mapping = {"G": "G", "S": "S", "D": "D"}
    reverse_ato_mapping = {
        pin: signal for signal, pins in ato_mapping.items() for pin in pins
    }

    transformed_lines = []
    for line in kicad_mod_content.splitlines():
        if line.strip().startswith("(pad"):
            parts = line.strip().split(" ")
            original_pin = parts[1]
            signal = reverse_ato_mapping.get("pin " + original_pin)
            if signal and signal in desired_mapping:
                parts[1] = desired_mapping[signal]  # Apply the transformation
            transformed_line = "\t" + " ".join(parts)  # Adding tab indent
        else:
            transformed_line = "\t" + line.strip()
        transformed_lines.append(transformed_line)
    return "\n".join(transformed_lines)


def extract_product_details(html):
    """
    Extracts product details from the given HTML string based on the provided structure.

    Args:
    html (str): A string containing the HTML content.

    Returns:
    dict: A dictionary containing the extracted product details.
    """
    # A dictionary to hold the extracted details
    details = {}

    # Regular expression pattern to match the details
    pattern = r"<dt[^>]*>(.*?)</dt>\s*<dd[^>]*>(.*?)</dd>"

    # Find all matches in the HTML content
    matches = re.findall(pattern, html, re.DOTALL)

    # Process each match
    for match in matches:
        key = match[0].strip()
        value = re.sub(
            "<[^<]+?>", "", match[1]
        ).strip()  # Remove any HTML tags from the value
        details[key] = value

    return details


def extract_attribute_data(html):
    """
    Extracts attribute data from the given HTML string and returns it as a dictionary.

    Args:
    html (str): A string containing the HTML content.

    Returns:
    dict: A dictionary containing the extracted attribute names and values.
    """
    # Define the pattern to match the specified structure
    pattern = (
        r'\{(?:\s*attribute_name_en:\s*"([^"]+)"\s*,\s*attribute_value_name:\s*"([^"]+)"|'
        + r'\s*attribute_value_name:\s*"([^"]+)"\s*,\s*attribute_name_en:\s*"([^"]+)")\s*\}'
    )

    # Find all matches in the HTML content
    matches = re.findall(pattern, html)

    # Create a dictionary from the matches
    extracted_data = {}
    for match in matches:
        # Match could be in either of the two capturing groups, depending on the order in HTML
        attribute_name = match[0] if match[0] else match[3]
        attribute_value = match[1] if match[1] else match[2]
        extracted_data[attribute_name] = attribute_value

    return extracted_data


def extract_prices(html_content):
    """
    Extracts pricing details from the given HTML content.

    Args:
    html_content (str): A string containing the HTML content.

    Returns:
    list of tuples: A list where each tuple contains the quantity and the corresponding price.
    """
    # Parse the HTML content
    soup = BeautifulSoup(html_content, "html.parser")

    # Extracting price information
    price_divs = soup.find_all("div", class_="flex items-center justify-between mt-14")
    prices = []

    for div in price_divs:
        # Extract quantity and price
        qty = div.find("span", class_="w-120 inline-block").text.strip()
        pr = div.find("span", class_="w-120 text-left").text.strip().replace("$", "")

        # Add to the list as a tuple
        prices.append((qty, float(pr)))

    return prices


def generate_uuid():
    """
    Generates a UUID.

    Returns:
    str: A UUID with ato- prefix.
    """
    return f"ato-{uuid.uuid4()}"


def calculate_physical_properties(value=None, tolerance=None, min_val=None, max_val=None, unit=None):
    """
    [Function definition from the previous response]
    """
    properties = {}

    if value is not None and tolerance is not None:
        # Calculate min and max values based on tolerance
        nominal = ureg.Quantity(value, unit)
        tolerance_val = nominal * tolerance / 100
        min_val = nominal - tolerance_val
        max_val = nominal + tolerance_val
    elif min_val is not None and max_val is not None:
        # Calculate nominal value and tolerance
        min_val = ureg.Quantity(min_val, unit)
        max_val = ureg.Quantity(max_val, unit)
        nominal = (min_val + max_val) / 2
        tolerance_val = max_val - nominal
    else:
        raise ValueError("Either value with tolerance or min and max values must be provided.")

    # Convert and format the unit
    if unit == 'uH':
        nominal = nominal.to(ureg.henry)
        tolerance_val = tolerance_val.to(ureg.henry)
        min_val = min_val.to(ureg.henry)
        max_val = max_val.to(ureg.henry)

    properties['nominal'] = nominal.magnitude
    properties['tolerance'] = tolerance_val.magnitude
    properties['min_val'] = min_val.magnitude
    properties['max_val'] = max_val.magnitude
    properties['tolerance_pct'] = (tolerance_val / nominal * 100).magnitude
    properties['unit'] = str(nominal.units)

    return properties


def get_html(lcsc_id):
    """
    Returns the HTML content for the specified LCSC ID.

    Args:
    lcsc_id (str): The LCSC ID for the product.

    Returns:
    str: A string containing the HTML content.
    """
    # Define the URL for the product
    url = f"https://jlcpcb.com/partdetail/{lcsc_id}"

    # Get the HTML content
    response = requests.get(url)
    return response.text

def extract_stock(html_string):
    # Regular expression pattern to find stock level
    stock_pattern = r'Available Order Qty: (\d+)'

    # Search for the pattern in the HTML string
    match = re.search(stock_pattern, html_string)

    # Extract and return stock level if found, else return 0
    if match:
        return int(match.group(1))
    else:
        return 0

def process_physical_attribute(value, unit, tolerance=None, min_val=None):
    ureg = UnitRegistry()
    quantity = ureg.Quantity(value, unit)

    # Convert the quantity to its base units
    quantity = quantity.to_base_units()

    if tolerance is not None:
        tolerance_val = quantity * tolerance / 100
        min_val = quantity - tolerance_val if min_val is None else ureg.Quantity(min_val, unit).to_base_units()
        max_val = quantity + tolerance_val
    else:
        min_val = ureg.Quantity(min_val, unit).to_base_units() if min_val is not None else quantity
        max_val = quantity

    return {
        'nominal': quantity.magnitude,
        'min_val': min_val.magnitude if min_val is not None else None,
        'max_val': max_val.magnitude,
        'tolerance': tolerance,
        'unit': str(quantity.units)
    }
# print(process_physical_attribute(10, 'uH', 10, 9))


In [ ]:
min_val, max_val = "5.45V~5.75V".split('~')
unit = extract_value_and_unit(min_val)[1]
# diode_data['forward_voltage'] = process_physical_attribute(max_val, unit, min_val=min_val)

In [155]:
import json
from firebase_admin import firestore, initialize_app
from firebase_functions import firestore_fn, https_fn
from pint import UnitRegistry

ureg = UnitRegistry()  # If you have custom unit definitions
# app = initialize_app()

IGNORED_PROPERTIES = ['designator_prefix', 'mpn']

firestore_client = firestore.client()


def get_component(request, firestore_client=firestore_client):
    try:
        req_json = request
        component_type = req_json.get("type", "").lower()
        if not component_type:
            raise ValueError("Component type is required.")

        physical_filters = {}
        property_filters = {"type": component_type}

        for key, value in req_json.items():
            if key in IGNORED_PROPERTIES or key in ["type"]:
                continue  # Skip ignored and separately handled properties

            if isinstance(value, dict) and "min_val" in value and "max_val" in value:
                base_min_val = convert_to_base_units(value["min_val"], value["unit"])
                base_max_val = convert_to_base_units(value["max_val"], value["unit"])
                physical_filters[key] = (base_min_val, base_max_val)
            else:
                property_filters[key] = value

        print(physical_filters)
        print(property_filters)

        components = query_components(firestore_client, component_type, physical_filters, property_filters)

        if not components:
            return json.dumps({"error": "No components found that match the criteria."}),

        best_component = select_best_component(components)

        return best_component


    except Exception as e:
        return str(e)

def query_components(firestore_client, component_type, physical_filters=None, property_filters=None):
    query = firestore_client.collection("component_data").where("type", "==", component_type)

    # Apply property filters
    for prop, value in property_filters.items():
        if prop != "type":  # 'type' is already included in the query
            query = query.where(prop, "==", value)

    results = query.limit(10).stream()
    components = [doc.to_dict() for doc in results]

    # Apply physical filters
    if physical_filters:
        print(physical_filters)
        filtered_components = [comp for comp in components if all(
            prop in comp and
            physical_filters[prop][0] <= comp.get(prop, {}).get('max_val', float('inf')) and
            physical_filters[prop][1] >= comp.get(prop, {}).get('min_val', 0)
            for prop in physical_filters
        )]

    return filtered_components


def convert_to_base_units(value, unit):
    quantity = ureg.Quantity(value, unit)
    base_unit = ureg.Quantity(1, unit).to_base_units().units
    return quantity.to(base_unit).magnitude

def select_best_component(components):
    in_stock_components = [comp for comp in components if comp.get("stock", 0) > 100]
    if in_stock_components:
        components = in_stock_components

    basic_parts = [comp for comp in components if comp.get("basic_part", False)]
    if basic_parts:
        components = basic_parts

    return min(components, key=lambda comp: comp.get("price_usd", float('inf')), default=None)


In [158]:
request ={
    "designator_prefix": "D",
    "type": "diode",
    "mpn": "generic_diode",
    "average_rectified_current": {
        "unit": "ampere",
        "min_val": 0,
        "max_val": 20,
        "nominal": 3.5,
        "tolerance": 2.5,
        "tolerance_pct": 71.42857142857143,
    },
}

get_component(request)

{'average_rectified_current': (0, 20)}

{'type': 'diode'}

{'average_rectified_current': (0, 20)}

{'description': '1A Single 1000V 1.1V@1A SMA(DO-214AC)  Diodes - General Purpose ROHS',
 'type': 'diode',
 'uuid': 'ato-0294c876-24a2-4390-b439-fc1edac3bda5',
 'stock': 136529,
 'price': 0.0074,
 'average_rectified_current': {'min_val': 0.9,
  'nominal': 1.0,
  'tolerance': 10,
  'unit': 'ampere',
  'max_val': 1.1},
 'reverse_leakage_current': {'min_val': 4.5,
  'nominal': 5.0,
  'tolerance': 10,
  'unit': 'microampere',
  'max_val': 5.5},
 'price_usd': 0.0074,
 'forward_voltage': {'min_val': 0.9900000000000001,
  'nominal': 1.1,
  'tolerance': 10,
  'unit': 'volt',
  'max_val': 1.2100000000000002},
 'footprint': {'kicad': 'ato-0294c876-24a2-4390-b439-fc1edac3bda5.kicad_mod'},
 'category': 'general_purpose',
 'lcsc_id': 'C115218',
 'package': 'SMA(DO-214AC)',
 'footprint_data': {'kicad': '\t(module easyeda2kicad:SMA_L4.4-W2.6-LS5.0-RD (layer F.Cu) (tedit 5DC5F6A4)\n\t(attr smd)\n\t(fp_text reference REF** (at 0 -4.0) (layer F.SilkS)\n\t(effects (font (size 1 1) (thickness 0.15)))\n\t)\